### Melon100 Chart
* 100곡의 노래의 제목과 SongId 추출해서 list에 저장
* 노래의 상세정보를 추출해서 list와 dict에 저장해서 json 파일로 저장하기
* json 파일을 load 하여 Pandas의 DataFrame에 저장하기
* DataFrame 객체를 DB에 Table에 저장하기

In [1]:
import requests
from bs4 import BeautifulSoup
import re

In [208]:
url = 'https://www.melon.com/chart/index.htm'
rec_hearder_dict = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'
}

res = requests.get(url,headers=rec_hearder_dict)

if res.ok:
    html = res.text
    soup = BeautifulSoup(html,'html.parser')
    # print(len(soup.select("div#tb_list tr a[href*='playSong']")))
    a_tags = soup.select("div#tb_list tr a[href*='playSong']")
    song_info_list = []
    #노래 100곡의 정보를 저장할 list
    for idx,a_tag in enumerate(a_tags,1):
        song_dict = {}
        #노래 1곡의 정보를 저장할 dict 선언 (제목, songid, url)
        song_title = a_tag.text
        href_value = a_tag['href']
        song_dict['song_title'] = song_title
        # print(href_value)
        # print(idx,song_title)
        #정규표현식 사용 사이트 regexr.com
        # href에 있는 songid의 값을 찾기위해
        mached = re.search(r'(\d+)\);',href_value) #정규표현식 re 라이브러리 사용 정규표현식 쓸때는 r''사용
        if mached :  ##songid가 있으면
            song_id = mached.group(1) ##group(0)은 ); 아 포함된거 group(1)은 숫자만 포함된거임!
            song_dict['song_id'] = song_id
            song_detail_url = f'https://www.melon.com/song/detail.htm?songId={song_id}'
            song_dict['song_detail_url'] = song_detail_url
            # print(song_id,song_detail_url)
            # print(song_dict)
        song_info_list.append(song_dict)



In [209]:
song_detail_list = []

for idx,song in enumerate(song_info_list,1):
    #위에서 저장한 url을 통해 정보를 get
    #노래 1곡의 상세정보(곡명,가수,좋아요,장르...)를 저장할 dict
    song_detail_dict = {}

    # print(song['song_detail_url'])
    song_detail_url = song['song_detail_url']

    res = requests.get(song_detail_url,headers=rec_hearder_dict)
    soup = BeautifulSoup(res.text,'html.parser')


    if res.ok:
        song_detail_dict['곡명'] = song['song_title']
        singer_span = soup.select("a[href*='goArtistDetail'] span")## a태그 밑에 span 태그
        print(idx,song['song_title'])
        if singer_span:
            song_detail_dict['가수'] = singer_span[0].text   ##singer_span에는 여러가지 span태그들이 존재함 그중 첫번째 것을 text화 시킴


        song_dd = soup.select("div.meta dd")

    if song_dd:
        song_detail_dict['앨범'] = song_dd[0].text
        song_detail_dict['발매일'] = song_dd[1].text
        song_detail_dict['장르'] = song_dd[2].text


        song_id = song['song_id']
        #좋아요 개수를 구하기 위해 request를 다시 날림!
        like_url = f'https://www.melon.com/commonlike/getSongLike.json?contsIds={song_id}'
        like_res = requests.get(like_url,headers=rec_hearder_dict)

        #개발자도구에서 network로가서 구함
        likes_count = like_res.json()['contsLike'][0]['SUMMCNT']


        song_detail_dict['좋아요'] = likes_count
    #노래가사
    #<div class = "lyric" id = "d_video_summary">
    lyric_div = soup.select("div#d_video_summary")
    if lyric_div:
        lyric_temp = lyric_div[0].text
        pattern = re.compile(r'[\r\n\t]') #정규표현식을 찾는 패턴을 만듬
        lyric = pattern.sub('',lyric_temp.strip()) # \r\n\t(특수문자)를 ''(empty string)으로 대체해라(substitute) 그리고 공백제거까지
        song_detail_dict['가사'] = lyric
    else:
        lyric = ''





    song_detail_list.append(song_detail_dict)




1 취중고백
2 사랑은 늘 도망가
3 호랑수월가
4 Step Back
5 ELEVEN
6 회전목마 (Feat. Zion.T, 원슈타인) (Prod. Slom)
7 리무진 (Feat. MINO) (Prod. GRAY)
8 Counting Stars (Feat. Beenzino)
9 Dreams Come True
10 눈이 오잖아(Feat.헤이즈)
11 다정히 내 이름을 부르면
12 Can't Control Myself
13 겨울잠
14 이제 나만 믿어요
15 SMILEY (Feat. BIBI)
16 STAY
17 만남은 쉽고 이별은 어려워 (Feat. Leellamarz) (Prod. TOIL)
18 신호등
19 Next Level
20 strawberry moon
21 Savage
22 Christmas Tree
23 드라마
24 그대라는 사치
25 다시 사랑한다면 (김필 Ver.)
26 별빛 같은 나의 사랑아
27 우리가 헤어져야 했던 이유
28 언덕나무
29 서랍
30 흰눈
31 잊었니
32 HERO
33 abcdefu
34 너를 생각해
35 끝사랑
36 Butter
37 OHAYO MY NIGHT
38 사랑이 아니었다고 말하지 마요
39 문득
40 네가 없는 밤 (Feat. ASH ISLAND) (Prod. GRAY)
41 Bk Love
42 불협화음 (Feat. AKMU) (Prod. GRAY)
43 너 아니면 안돼
44 Dynamite
45 바라만 본다
46 Permission to Dance
47 Child
48 여름비
49 My Universe
50 Weekend
51 쉬어 (Feat. MINO) (Prod. GRAY)
52 낙하 (with 아이유)
53 Celebrity
54 Wake Up (Prod. 코드 쿤스트)
55 그게 더 편할 것 같아 (N번째 연애 X 멜로망스)
56 밤하늘의 별을(2020)
57 라일락
58 헤픈 우연
59 사이렌 Remix (Feat. UNEDUCATED KID, Paul Blanco)
60 이별후회
61 Bad H

In [125]:
## json으로 저장하기
import json


with open('data/songs.json','w',encoding='UTF-8') as file :
    file.write(json.dumps(song_detail_list,ensure_ascii=False)) #json 파일 쉽게 보려 http://jsonviewer.stack.hu 여기로가서봄

### Pandas 의 DataFrame 객체 사용하기

In [131]:
import pandas as pd
##song_detail_list를 읽이서 DataFrame 객체를 생성하는 방법
#DataFrame 객체 생성
song_df2 = pd.DataFrame(columns=['곡명','가수','앨범','발매일','장르','좋아요','가사'])
for song_detail in song_detail_list:
    series_obj = pd.Series(song_detail)
    song_df2 = song_df2.append(series_obj,ignore_index=True)


In [137]:
import pandas as pd
song_df = pd.read_json('data/songs.json')
print('shape',song_df.shape) ##몇행 몇열인가?
print('columns',song_df.columns)
print('index',song_df.index)
print('values',song_df.values[0])

shape (100, 7)
columns Index(['곡명', '가수', '앨범', '발매일', '장르', '좋아요', '가사'], dtype='object')
index RangeIndex(start=0, stop=100, step=1)
values ['사랑은 늘 도망가' '임영웅' '신사와 아가씨 OST Part.2' '2021.10.11' '발라드, 국내드라마' 123992
 '눈물이 난다 이 길을 걸으면그 사람 손길이 자꾸 생각이 난다붙잡지 못하고 가슴만 떨었지내 아름답던 사람아사랑이란 게 참 쓰린 거더라잡으려 할수록 더 멀어지더라이별이란 게 참 쉬운 거더라내 잊지 못할 사람아사랑아 왜 도망가수줍은 아이처럼행여 놓아버릴까 봐꼭 움켜쥐지만그리움이 쫓아 사랑은 늘 도망가잠시 쉬어가면 좋을 텐데바람이 분다 옷깃을 세워도차가운 이별의 눈물이 차올라잊지 못해서 가슴에 사무친내 소중했던 사람아사랑아 왜 도망가수줍은 아이처럼행여 놓아버릴까 봐꼭 움켜쥐지만그리움이 쫓아 사랑은 늘 도망가잠시 쉬어가면 좋을 텐데기다림도 애태움도 다 버려야 하는데무얼 찾아 이 길을 서성일까무얼 찾아 여기 있나사랑아 왜 도망가수줍은 아이처럼행여 놓아버릴까 봐꼭 움켜쥐지만그리움이 쫓아 사랑은 늘 도망가잠시 쉬어가면 좋을 텐데잠시 쉬어가면 좋을 텐데']


In [139]:
song_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   곡명      100 non-null    object
 1   가수      100 non-null    object
 2   앨범      100 non-null    object
 3   발매일     100 non-null    object
 4   장르      100 non-null    object
 5   좋아요     100 non-null    int64 
 6   가사      100 non-null    object
dtypes: int64(1), object(6)
memory usage: 5.6+ KB


In [144]:
#가수 컬럼의 값을 선택하기

print(type(song_df['가수']))
print(song_df['가수'])

<class 'pandas.core.series.Series'>
0              임영웅
1       김민석 (멜로망스)
2               탑현
3     GOT the beat
4         sokodomo
          ...     
95             조정석
96     케이시 (Kassy)
97             MC몽
98             토요태
99       BE'O (비오)
Name: 가수, Length: 100, dtype: object


In [154]:
#가수 컬럼의 값별로 Row Counting
song_df

print(song_df['가수'].value_counts())

임영웅                    10
아이유                     7
방탄소년단                   6
BE'O (비오)               5
aespa                   3
                       ..
Coldplay                1
아넌딜라이트(Anandelight)     1
김민석 (멜로망스)              1
쿠기 (Coogie)             1
토요태                     1
Name: 가수, Length: 65, dtype: int64


In [155]:
#unoque 합 중복제거
print(song_df['가수'].unique())

['임영웅' '김민석 (멜로망스)' '탑현' 'GOT the beat' 'sokodomo' 'IVE (아이브)' "BE'O (비오)"
 'aespa' '이무진' '태연 (TAEYEON)' '아이유' '경서예지' '베이식 (Basick)' 'The Kid LAROI'
 'YENA (최예나)' 'V' '이승윤' '비비 (BIBI)' '10CM' '먼데이 키즈 (Monday Kiz)' '방탄소년단'
 '주시크 (Joosiq)' '이예준' 'GAYLE' '마크 (MARK)' '디핵 (D-Hack)' 'Mudd the student'
 '샘김 (Sam Kim)' 'MSG워너비(M.O.M)' 'Coldplay' '아넌딜라이트(Anandelight)'
 'AKMU (악뮤)' '개코' '멜로망스' '쿠기 (Coogie)' '김나영' '경서' '헤이즈 (Heize)'
 'Ed Sheeran' '호미들' '황인욱' '마크툽 (MAKTUB)' '폴킴' '프로미스나인' '선예'
 'Justin Bieber' '전상근' '성시경' '정동하' 'ASH ISLAND' '브레이브걸스'
 '오마이걸 (OH MY GIRL)' '송이한' 'TOIL' '산들' '케이윌' '휘인 (Whee In)' 'Jawsh 685'
 'Red Velvet (레드벨벳)' 'Kep1er (케플러)' '잠골버스' '조정석' '케이시 (Kassy)' 'MC몽' '토요태']


In [156]:
#가사 empty string check

print(song_df.loc[song_df['가사'] == ''])

Empty DataFrame
Columns: [곡명, 가수, 앨범, 발매일, 장르, 좋아요, 가사]
Index: []


In [169]:
#가수가 방탄소년단 인 행을 선택해보자
# print(song_df['가수'] == '방탄소년단')
##만족하는 행을 선택할때 loc 사용

# print(song_df.loc[song_df['가수'] == '방탄소년단'])


def searchSinger(singer_name):
    print(song_df['곡명'].loc[song_df['가수'] == singer_name])


searchSinger('임영웅')

0              사랑은 늘 도망가
10             이제 나만 믿어요
17    다시 사랑한다면 (김필 Ver.)
18               그대라는 사치
23          별빛 같은 나의 사랑아
25                   잊었니
28                  HERO
29                   끝사랑
32               Bk Love
42            계단말고 엘리베이터
Name: 곡명, dtype: object


### 특정 행과 열을 선택하기
* loc[] , iloc[] 사용한다.
1. Slicing 을 사용해서 구간을 주어서 행과 열을 선택
2. [ ] (list)를 사용해서 특정행과 열을 선택
3. 조건식을 만족하는 행과 열을 선택

In [171]:
#Slicing 인덱스가 0부터 5까지의 행과 모든 열을 선택하기
song_df.loc[0:5] # == song_df.loc[0:5,:]


,곡명,가수,앨범,발매일,장르,좋아요,가사
1,취중고백,김민석 (멜로망스),취중고백,2021.12.19,발라드,68692,뭐하고 있었니 늦었지만잠시 나올래너의 집 골목에 있는놀이터에 앉아 있어친구들 만나서...
2,호랑수월가,탑현,호랑수월가,2022.01.15,발라드,35302,흐르는 저 하늘을 물어 채는 범처럼태산에 날아들어 숨어드는 새처럼동산을 뛰고 뛰어가...
3,Step Back,GOT the beat,Step Back,2022.01.03,댄스,65667,You must step back어델 어델 봐너 감히 누구라고 날 제껴이쯤에서 물러...
4,"회전목마 (Feat. Zion.T, 원슈타인) (Prod. Slom)",sokodomo,쇼미더머니 10 Episode 2,2021.11.13,랩/힙합,186554,내가 슬플 때마다이 노래가 찾아와세상이 둥근 것처럼 우린 동글동글인생은 회전목마우린...
5,ELEVEN,IVE (아이브),ELEVEN,2021.12.01,댄스,103239,따분한 나의 눈빛이무표정했던 얼굴이널 보며 빛나고 있어널 담은 눈동자는 odd내 안...


In [174]:
#slicing : 인덱스가 0부터 10까지의 행과 모든 열을 선택하기, 1개의 행을 skip
song_df.loc[0:10:2] #0,2,4,6,8,10 이 출력 == song_df.loc[0:10:2,:] , [start,stop,step]

,곡명,가수,앨범,발매일,장르,좋아요,가사
0,사랑은 늘 도망가,임영웅,신사와 아가씨 OST Part.2,2021.10.11,"발라드, 국내드라마",123992,눈물이 난다 이 길을 걸으면그 사람 손길이 자꾸 생각이 난다붙잡지 못하고 가슴만 떨...
2,호랑수월가,탑현,호랑수월가,2022.01.15,발라드,35302,흐르는 저 하늘을 물어 채는 범처럼태산에 날아들어 숨어드는 새처럼동산을 뛰고 뛰어가...
4,"회전목마 (Feat. Zion.T, 원슈타인) (Prod. Slom)",sokodomo,쇼미더머니 10 Episode 2,2021.11.13,랩/힙합,186554,내가 슬플 때마다이 노래가 찾아와세상이 둥근 것처럼 우린 동글동글인생은 회전목마우린...
6,리무진 (Feat. MINO) (Prod. GRAY),BE'O (비오),쇼미더머니 10 Episode 3,2021.11.20,랩/힙합,162366,까만 리무진 보며꿈을 키웠지언젠가는 나도 저걸 갖게 될 거야커다란 리무진에서 내가 ...
8,Dreams Come True,aespa,Dreams Come True - SM STATION,2021.12.20,댄스,74895,"Uh, You wanna feel the vibeThat I’m feeling an..."
10,이제 나만 믿어요,임영웅,내일은 미스터트롯 우승자 특전곡,2020.04.03,성인가요,146420,무얼 믿은 걸까 부족했던 내게서나조차 못 믿던 내게 여태 머문 사람무얼 봤던 걸까 ...


In [175]:
#Slicing 인덱스가 0부터 5까지의 행에서 곡명부터 장르까지의 열을 선택하기
song_df.loc[0:5,'곡명':'장르']

,곡명,가수,앨범,발매일,장르
0,사랑은 늘 도망가,임영웅,신사와 아가씨 OST Part.2,2021.10.11,"발라드, 국내드라마"
1,취중고백,김민석 (멜로망스),취중고백,2021.12.19,발라드
2,호랑수월가,탑현,호랑수월가,2022.01.15,발라드
3,Step Back,GOT the beat,Step Back,2022.01.03,댄스
4,"회전목마 (Feat. Zion.T, 원슈타인) (Prod. Slom)",sokodomo,쇼미더머니 10 Episode 2,2021.11.13,랩/힙합
5,ELEVEN,IVE (아이브),ELEVEN,2021.12.01,댄스


## 여러개의 행과 열을 선택

In [176]:
song_df.loc[[0,4,7,10]] ## 0,4,7,10번째 행을 선택

,곡명,가수,앨범,발매일,장르,좋아요,가사
0,사랑은 늘 도망가,임영웅,신사와 아가씨 OST Part.2,2021.10.11,"발라드, 국내드라마",123992,눈물이 난다 이 길을 걸으면그 사람 손길이 자꾸 생각이 난다붙잡지 못하고 가슴만 떨...
4,"회전목마 (Feat. Zion.T, 원슈타인) (Prod. Slom)",sokodomo,쇼미더머니 10 Episode 2,2021.11.13,랩/힙합,186554,내가 슬플 때마다이 노래가 찾아와세상이 둥근 것처럼 우린 동글동글인생은 회전목마우린...
7,Counting Stars (Feat. Beenzino),BE'O (비오),Counting Stars,2021.12.12,랩/힙합,147754,Counting stars밤하늘에 펄Better than your LVYour LV...
10,이제 나만 믿어요,임영웅,내일은 미스터트롯 우승자 특전곡,2020.04.03,성인가요,146420,무얼 믿은 걸까 부족했던 내게서나조차 못 믿던 내게 여태 머문 사람무얼 봤던 걸까 ...


## 여러개의 행과 곡명 가수 앨범 장르 선택

In [177]:
song_df.loc[[0,4,7,10],['곡명','가수','앨범','장르']]

,곡명,가수,앨범,장르
0,사랑은 늘 도망가,임영웅,신사와 아가씨 OST Part.2,"발라드, 국내드라마"
4,"회전목마 (Feat. Zion.T, 원슈타인) (Prod. Slom)",sokodomo,쇼미더머니 10 Episode 2,랩/힙합
7,Counting Stars (Feat. Beenzino),BE'O (비오),Counting Stars,랩/힙합
10,이제 나만 믿어요,임영웅,내일은 미스터트롯 우승자 특전곡,성인가요


In [179]:
# 여러개의 0,4,7,10 행과 좋아요 열만 선택
song_df.loc[[0,4,7,10],['좋아요']]

,좋아요
0,123992
4,186554
7,147754
10,146420


In [180]:
#10부터 20까지의 행과 곡명 가수 좋아요 열을 선택
song_df.loc[10:20,['곡명','가수','좋아요']]

,곡명,가수,좋아요
10,이제 나만 믿어요,임영웅,146420
11,Can't Control Myself,태연 (TAEYEON),47546
12,겨울잠,아이유,89259
13,다정히 내 이름을 부르면,경서예지,109308
14,신호등,이무진,256451
15,만남은 쉽고 이별은 어려워 (Feat. Leellamarz) (Prod. TOIL),베이식 (Basick),107154
16,STAY,The Kid LAROI,202791
17,다시 사랑한다면 (김필 Ver.),임영웅,40774
18,그대라는 사치,임영웅,36832
19,Next Level,aespa,235363


In [184]:
#장르가 댄스인 행과 모든열을 선택하세요
song_df.loc[song_df['장르'] == '댄스']

,곡명,가수,앨범,발매일,장르,좋아요,가사
3,Step Back,GOT the beat,Step Back,2022.01.03,댄스,65667,You must step back어델 어델 봐너 감히 누구라고 날 제껴이쯤에서 물러...
5,ELEVEN,IVE (아이브),ELEVEN,2021.12.01,댄스,103239,따분한 나의 눈빛이무표정했던 얼굴이널 보며 빛나고 있어널 담은 눈동자는 odd내 안...
8,Dreams Come True,aespa,Dreams Come True - SM STATION,2021.12.20,댄스,74895,"Uh, You wanna feel the vibeThat I’m feeling an..."
19,Next Level,aespa,Next Level,2021.05.17,댄스,235363,I’m on the Next Level Yeah절대적 룰을 지켜내 손을 놓지 말아결...
21,SMILEY (Feat. BIBI),YENA (최예나),ˣ‿ˣ (SMiLEY),2022.01.17,댄스,44510,울지 마 울지 마어린아이같이웃는 게 웃는 게이기는 거라구Youre so cuteYo...
26,Savage,aespa,Savage - The 1st Mini Album,2021.10.05,댄스,139537,Oh my gosh!Don't you know I’m a Savage?I’m a K...
35,Butter,방탄소년단,Butter,2021.05.21,댄스,263254,Smooth like butterLike a criminal undercoverGo...
41,Dynamite,방탄소년단,Dynamite (DayTime Version),2020.08.24,댄스,428691,Cos ah ahI’m in the stars tonightSo watch me b...
45,Permission to Dance,방탄소년단,Butter / Permission to Dance,2021.07.09,댄스,195015,It’s the thought of being youngWhen your heart...
50,Weekend,태연 (TAEYEON),Weekend,2021.07.06,댄스,159196,가장 가까운 바다혼자만의 영화관그냥 이끌리는 대로 해도 괜찮으니까Every morn...


In [197]:
#장르가 댄스인 행과 곡명부터 장르까지
song_df.loc[song_df['장르']=='댄스','곡명':'장르'].reset_index(drop='index')  #조건을 걸고 출력할때 다시 인덱스를 없애기 위해 reset을 한다
# reset을 하면 기존 인덱스가 새로운 컬럼으로 들어온다 하지만 reset인에 drop = 'index' 를 넣으면 인덱스가 없어진다.


,곡명,가수,앨범,발매일,장르
0,Step Back,GOT the beat,Step Back,2022.01.03,댄스
1,ELEVEN,IVE (아이브),ELEVEN,2021.12.01,댄스
2,Dreams Come True,aespa,Dreams Come True - SM STATION,2021.12.20,댄스
3,Next Level,aespa,Next Level,2021.05.17,댄스
4,SMILEY (Feat. BIBI),YENA (최예나),ˣ‿ˣ (SMiLEY),2022.01.17,댄스
5,Savage,aespa,Savage - The 1st Mini Album,2021.10.05,댄스
6,Butter,방탄소년단,Butter,2021.05.21,댄스
7,Dynamite,방탄소년단,Dynamite (DayTime Version),2020.08.24,댄스
8,Permission to Dance,방탄소년단,Butter / Permission to Dance,2021.07.09,댄스
9,Weekend,태연 (TAEYEON),Weekend,2021.07.06,댄스


In [195]:

song_df.loc[song_df['좋아요'] > song_df['좋아요'].mean()]

117366.31


,곡명,가수,앨범,발매일,장르,좋아요,가사
0,사랑은 늘 도망가,임영웅,신사와 아가씨 OST Part.2,2021.10.11,"발라드, 국내드라마",123992,눈물이 난다 이 길을 걸으면그 사람 손길이 자꾸 생각이 난다붙잡지 못하고 가슴만 떨...
4,"회전목마 (Feat. Zion.T, 원슈타인) (Prod. Slom)",sokodomo,쇼미더머니 10 Episode 2,2021.11.13,랩/힙합,186554,내가 슬플 때마다이 노래가 찾아와세상이 둥근 것처럼 우린 동글동글인생은 회전목마우린...
6,리무진 (Feat. MINO) (Prod. GRAY),BE'O (비오),쇼미더머니 10 Episode 3,2021.11.20,랩/힙합,162366,까만 리무진 보며꿈을 키웠지언젠가는 나도 저걸 갖게 될 거야커다란 리무진에서 내가 ...
7,Counting Stars (Feat. Beenzino),BE'O (비오),Counting Stars,2021.12.12,랩/힙합,147754,Counting stars밤하늘에 펄Better than your LVYour LV...
10,이제 나만 믿어요,임영웅,내일은 미스터트롯 우승자 특전곡,2020.04.03,성인가요,146420,무얼 믿은 걸까 부족했던 내게서나조차 못 믿던 내게 여태 머문 사람무얼 봤던 걸까 ...
14,신호등,이무진,신호등,2021.05.14,록/메탈,256451,이제야 목적지를 정했지만가려한 날 막아서네 난 갈 길이 먼데새빨간 얼굴로 화를 냈던...
16,STAY,The Kid LAROI,Stay,2021.07.09,POP,202791,I do the same thing I told youthat I never wou...
19,Next Level,aespa,Next Level,2021.05.17,댄스,235363,I’m on the Next Level Yeah절대적 룰을 지켜내 손을 놓지 말아결...
20,strawberry moon,아이유,strawberry moon,2021.10.19,록/메탈,179842,달이 익어가니 서둘러 젊은 피야민들레 한 송이 들고사랑이 어지러이 떠다니는 밤이야날...
26,Savage,aespa,Savage - The 1st Mini Album,2021.10.05,댄스,139537,Oh my gosh!Don't you know I’m a Savage?I’m a K...


In [200]:
song_df['좋아요'].max()
song_df['좋아요'].min()
song_df['좋아요'].median()

79262.0

In [207]:
like = song_df.loc[song_df['좋아요'] >= song_df['좋아요'].mean(),['곡명','가수','좋아요']]
like.sort_values(by = '좋아요',ascending=False).reset_index(drop=True)

,곡명,가수,좋아요
0,봄날,방탄소년단,555844
1,작은 것들을 위한 시 (Boy With Luv) (Feat. Halsey),방탄소년단,431667
2,Dynamite,방탄소년단,428691
3,"모든 날, 모든 순간 (Every day, Every Moment)",폴킴,385892
4,"어떻게 이별까지 사랑하겠어, 널 사랑하는 거지",AKMU (악뮤),373839
5,Blueming,아이유,328420
6,Celebrity,아이유,312341
7,롤린 (Rollin'),브레이브걸스,290787
8,Butter,방탄소년단,263254
9,신호등,이무진,256451
